# Mitochondrial Gene Identification

This notebook integrates steps to identify mitochondrial genes in a new species using a reference species and analyze expression correlation.

## 1. Preprocess GTF to keep longest transcript

In [ ]:

import gffutils
import pandas as pd

def keep_longest_transcripts(gtf_path, out_gtf, db_path=None):
    """Keep only the longest transcript for each gene in a GTF file."""
    db_path = db_path or gtf_path + '.db'
    db = gffutils.create_db(gtf_path, dbfn=db_path, force=True,
                            keep_order=True, disable_infer_genes=True,
                            disable_infer_transcripts=True)
    tx_lengths = []
    for tx in db.features_of_type('transcript', order_by='start'):
        exons = list(db.children(tx, featuretype='exon', order_by='start'))
        length = sum(e.end - e.start + 1 for e in exons)
        gene_id = tx.attributes['gene_id'][0]
        tx_lengths.append((tx.id, gene_id, length))
    df = pd.DataFrame(tx_lengths, columns=['transcript_id','gene_id','length'])
    longest_tx = df.sort_values('length').drop_duplicates('gene_id', keep='last')
    with open(out_gtf, 'w') as out:
        for tx_id in longest_tx.transcript_id:
            for feature in db.children(tx_id, featuretype='exon', order_by='start', level=1):
                out.write(str(feature) + '
')


Use the function for mouse and species X:

In [ ]:

# keep_longest_transcripts('mouse.gtf', 'mouse_longest.gtf')
# keep_longest_transcripts('speciesX.gtf', 'speciesX_longest.gtf')


## 2. Find reciprocal best BLAST hits

In [ ]:

%%bash
# Build BLAST databases and run reciprocal BLAST searches
# Adjust file paths accordingly

makeblastdb -in speciesX_longest.fa -dbtype nucl
blastn -query mouse_longest.fa -db speciesX_longest.fa        -out mouse_vs_X.tsv -outfmt 6 -evalue 1e-5 -max_target_seqs 1

makeblastdb -in mouse_longest.fa -dbtype nucl
blastn -query speciesX_longest.fa -db mouse_longest.fa        -out X_vs_mouse.tsv -outfmt 6 -evalue 1e-5 -max_target_seqs 1


In [ ]:

import pandas as pd

m_to_x = pd.read_csv('mouse_vs_X.tsv', sep='	', header=None,
                     usecols=[0,1], names=['mouse','X'])
x_to_m = pd.read_csv('X_vs_mouse.tsv', sep='	', header=None,
                     usecols=[0,1], names=['X','mouse'])
rbh = m_to_x.merge(x_to_m, on=['mouse','X'])
mouse_mt = pd.read_csv('mouse_mt_genes.txt', header=None, names=['mouse'])
rbh_mt = rbh.merge(mouse_mt, on='mouse')
rbh_mt.to_csv('X_mito_candidate.tsv', sep='	', index=False)


## 3. Map candidate genes to contigs or chromosomes

In [ ]:

import pandas as pd

gtf_cols = ['seqname','source','feature','start','end','score','strand','frame','attribute']
gtf_df = pd.read_csv('speciesX_longest.gtf', sep='	', comment='#', names=gtf_cols)
gtf_df['gene_id'] = gtf_df.attribute.str.extract('gene_id "([^"]+)"')

cand = pd.read_csv('X_mito_candidate.tsv', sep='	')
result = gtf_df[gtf_df.gene_id.isin(cand['X'])]
result[['seqname','gene_id']].drop_duplicates().groupby('seqname')['gene_id']       .apply(list).to_csv('contig_gene_mapping.txt')


## 4. Expression correlation in scRNA-seq data

In [ ]:

import scanpy as sc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

adata = sc.read_h5ad('speciesX_scRNA.h5ad')
cand = pd.read_csv('X_mito_candidate.tsv', sep='	')
genes = cand['X'].tolist()
adata_mt = adata[:, [g for g in genes if g in adata.var_names]]

expr = adata_mt.X.toarray() if hasattr(adata_mt.X, 'toarray') else adata_mt.X
corr = np.corrcoef(expr.T)

sns.clustermap(corr, cmap='vlag', center=0, xticklabels=genes, yticklabels=genes)
plt.show()
